In [1]:
import torch
import pandas as pd
import numpy as np

In [ ]:
df_tr = pd.read_csv('train.csv')
df_tr = df_tr.drop(df_tr[df_tr.MISSING_DATA == True].index)
df_tr = df_tr.drop(df_tr[df_tr.POLYLINE=='[]'].index)

In [ ]:
POLYLINE = df_tr.iloc[1600000:]['POLYLINE']
POLYLINE.to_csv('POLYLINE.csv')
del df_tr

In [2]:
POLYLINE = pd.read_csv('POLYLINE.csv')
POLYLINE = POLYLINE.drop(['Unnamed: 0'],axis=1)
POLYLINE['index'] = POLYLINE.index
POLYLINE.head()

,POLYLINE,index
0,"[[-8.615511,41.140755],[-8.615493,41.140764],[...",0
1,"[[-8.606511,41.144679],[-8.606718,41.144769],[...",1
2,"[[-8.583849,41.163129],[-8.584425,41.16294],[-...",2
3,"[[-8.64954,41.167269],[-8.649702,41.166702],[-...",3
4,"[[-8.608644,41.147748],[-8.6085,41.14773],[-8....",4


In [3]:
test_features=torch.load('test_set.pt')
test_labels = torch.load('test_label.pt')

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class MLP(nn.Module):
    def __init__(self, input_features):
        super(MLP, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(in_features=input_features,out_features=1)
        )
        

    def forward(self, x):
        x=x.to(torch.float32)
        x=self.layer1(x)
        return x


In [5]:
model = MLP(14)
model.load_state_dict(torch.load('linear.pth'))
criterion = nn.MSELoss()

In [10]:
def get_loss(index):
    input = test_features[index]
    out = model(input)
    loss = torch.sqrt(criterion(out, test_features[index]))
    return loss.detach().numpy().flatten()[0]

In [11]:
POLYLINE['loss'] = POLYLINE['index'].apply(get_loss)

In [12]:
POLYLINE.head()

,POLYLINE,index,loss
0,"[[-8.615511,41.140755],[-8.615493,41.140764],[...",0,650.484375
1,"[[-8.606511,41.144679],[-8.606718,41.144769],[...",1,553.842590
2,"[[-8.583849,41.163129],[-8.584425,41.16294],[-...",2,583.422729
3,"[[-8.64954,41.167269],[-8.649702,41.166702],[-...",3,704.650452
4,"[[-8.608644,41.147748],[-8.6085,41.14773],[-8....",4,751.890930


In [13]:
highest = POLYLINE.nlargest(10, 'loss')

In [14]:
highest

,POLYLINE,index,loss
24457,"[[-8.629308,41.145336],[-8.629551,41.145462],[...",24457,994.816284
28193,"[[-8.606979,41.150079],[-8.607132,41.150097],[...",28193,994.816284
71226,"[[-8.610264,41.151645],[-8.610237,41.151636],[...",71226,994.816284
76018,"[[-8.629902,41.154831],[-8.629911,41.154768],[...",76018,994.816284
99406,"[[-8.585631,41.148531],[-8.585631,41.148531],[...",99406,994.816284
38011,"[[-8.609958,41.140836],[-8.61003,41.140908],[-...",38011,984.361023
41858,"[[-8.688447,41.173749],[-8.688483,41.173758],[...",41858,984.361023
49719,"[[-8.685081,41.164308],[-8.685072,41.164308],[...",49719,984.361023
66330,"[[-8.585631,41.148594],[-8.585694,41.148855],[...",66330,984.361023
72045,"[[-8.57349,41.15754],[-8.573526,41.157612],[-8...",72045,984.361023


In [24]:
import folium
for idx in range(10):
    Positions = row['POLYLINE'].strip('][').split('],[')
    Longitudes = []
    Latitudes = []
    points = [(float(position.split(',')[1]),float(position.split(',')[0])) for position in Positions]
    m = folium.Map()
    # add marker for the start and ending points
    for point in [points[0], points[-1]]:
      folium.Marker(point).add_to(m)
    # add the lines
    folium.PolyLine(points, weight=5, opacity=1).add_to(m)
    m.save(f'./routes/route_map{idx}.html')